In [1]:
!export PYTHONPATH='/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/PythonNlpResearch:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Calculations:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Classifiers:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Classifiers/RegEx:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Classifiers/Trees:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Clustering:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/CodeGen:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/CollectionsHelper:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Data:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Data/Chicago:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Data/CoralBleaching:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Data/CoralBleachingWordTagger:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Data/SkinCancerWordTagger:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Data/GlobalWarming:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Data/GlobalWarmingAnnotated:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Data/GoogleNGrams:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/DeepLearning:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/DeepLearning/DeepBeliefNetwork:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/DeepLearning/Examples:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/DeepLearning/Examples/DeepAutoEncoder:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/DeepLearning/Examples/DeepAutoEncoder_MichaelNielsen:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/DeepLearning/MyCode:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/DeepLearning/MyCode/RecursiveAutoEncoder:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/DeepLearning/WordVectors:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Experiments:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Experiments/Chicago:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Experiments/CoralBleaching:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Experiments/GlobalWarming:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Experiments/GlobalWarming/Causal:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Experiments/GlobalWarming/WordClustering:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/FeatureExtraction/Text:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Frequency:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/LanguageModel:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Results:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Tagging:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/VectorSpace:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/VectorSpace/WordVectors:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/VectorSpace/WordVectors/Word2Vec:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/_Scratch/Sub:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Synonyms:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/DeepLearning/MyCode/:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/MontyLingua:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Experiments/CoralBleachingWordTagger/WindowBasedClassifier:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Experiments/Tagging:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Experiments/CoralBleachingWordTagger/DeepNN'

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pymongo
from pprint import pprint

In [11]:
import sys
sys.path.append("/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/notebooks/Causal Model/src/")
from causal_model_features import CausalModelType, distance_between, is_forward_relation, build_cb_causal_model, build_sc_causal_model
cb_mdl = build_cb_causal_model()
sc_mdl = build_sc_causal_model()

In [3]:
client = pymongo.MongoClient()
db = client.metrics_causal

In [4]:
def __precision__(tp, fp, fn):
    if tp + fp <= 0:
        return 0.0
    return tp / (tp + fp)

def __recall__(tp, fp, fn):
    if tp + fn <= 0:
        return 0.0
    return tp / (tp + fn)

def __f_beta__(r, p, beta):
    if r + p <= 0.0:
        return 0.0
    beta_squared = beta * beta
    #Harmonic mean
    return ((1.0 + beta_squared) * r * p) / (beta_squared * (r + p))

def rpf1a_from_tp_fp_tn_fn(tp, fp, tn, fn):
    r = __recall__(tp, fp, fn)
    p = __precision__(tp, fp, fn)
    f1 = __f_beta__(r, p, 1.0)

    a = (tp + tn) / (tp + fp + tn + fn)
    return (r, p, f1, a)

In [41]:
from collections import defaultdict, OrderedDict
def coll2algo(coll):
    if "RNN" in coll:
        return "RNN"
    if "STACKED" in coll:
        return "Stacking"
    if "PARSER" in coll:
        return "Parser"
    return ""

def compute_metrics_for_relations(collection, relns=None):
    rows = list(db[collection].find())
    assert len(rows) == 1
    row = rows[0]
    tally = defaultdict(int)
    for k,val in row.items():
        if "->" in k and (not relns or k in relns):
            for prop in "tp,tn,fp,fn".split(","):
                tally[prop] += val[prop]
                
    (rec, prec, f1, acc) = rpf1a_from_tp_fp_tn_fn(tally["tp"],tally["fp"],tally["tn"],tally["fn"])
    return dict(OrderedDict({
        "algo": coll2algo(collection),
        "recall": rec,
        "precision": prec,
        "f1_score": f1,
        "accuracy": acc
    }))

In [42]:
def round_data(df, places=3):
    df_copy = df.copy()
    fmt_str = "{0:." + str(places) + "f}"
    cols = set([v for v in df_copy.columns.values if "micro_" in v])
    for c in cols:
        df_copy[c] = df[c].apply(lambda d: fmt_str.format(d))  
    return df_copy

## Get All Crels

In [43]:
cb_crels = set()
row = list(db["CR_CB_TAGGING_VD_MOST_COMMON_TAG_RNN"].find())[0]
for k,v in row.items():
    if "->" in k:
        cb_crels.add(k)
len(cb_crels)

86

In [44]:
sc_crels = set()
row = list(db["CR_SC_TAGGING_VD_MOST_COMMON_TAG_RNN"].find())[0]
for k,v in row.items():
    if "->" in k:
        sc_crels.add(k)
len(sc_crels)

49

In [45]:
from causal_model_features import __nodes_between__
__nodes_between__("1", "11", cb_mdl)

-1

In [46]:
def partition_rels(crels):
    val = set()
    inv = set()
    to50 = set()
    adj = set()
    for crel in crels:
        a,b = crel.replace("Causer:","").replace("Result:","").split("->")
        if is_forward_relation(a,b,cb_mdl):
            if b == "50":
                to50.add(crel)
            if __nodes_between__(a,b, cb_mdl) == 1:
                adj.add(crel)
            val.add(crel)
        else:
            inv.add(crel)
    return {
        "valid": val, "invalid": inv, 
        "to 50": to50,
        "adjacent": adj
    }

cb_part = partition_rels(cb_crels)
sc_part = partition_rels(sc_crels)

In [47]:
cb_collections = [
    "CR_CB_TAGGING_VD_MOST_COMMON_TAG_RNN", 
    "CR_CB_STACKED_VD",
    "CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_VD"
]
test_collections_cb = ["TEST_" + c.replace("HYPER_PARAM_","") for c in cb_collections]
test_collections_sc = [c.replace("_CB","_SC") for c in test_collections_cb]

In [54]:
def analyze_performance_by_partition(part2relations, colls):
    rows_by_parttn = defaultdict(dict)
    for parttn, crels in part2relations.items():
        for coll in colls:
            metrics = compute_metrics_for_relations(coll, crels)
            d = rows_by_parttn[parttn]
            d["Partition"] = parttn
            d[metrics["algo"]] = metrics["f1_score"]
    return pd.DataFrame(rows_by_parttn.values())

analyze_performance_by_partition(cb_part, test_collections_cb)

,Partition,RNN,Stacking,Parser
0,valid,0.688987,0.717489,0.746667
1,invalid,0.148148,0.242424,0.360656
2,to 50,0.686154,0.725240,0.750000
3,adjacent,0.778723,0.797441,0.799172


In [55]:
analyze_performance_by_partition(sc_part, test_collections_sc)

,Partition,RNN,Stacking,Parser
0,valid,0.804620,0.774584,0.798241
1,invalid,0.640000,0.600000,0.671429
2,to 50,0.808219,0.768817,0.798646
3,adjacent,0.836299,0.805604,0.843806


In [56]:
rows = []
for c in test_collections_cb:
    d = compute_metrics_for_relations(c)
    rows.append(d)
pd.DataFrame(rows)

,algo,recall,precision,f1_score,accuracy
0,RNN,0.656093,0.698046,0.676420,0.997720
1,Stacking,0.674457,0.735883,0.703833,0.997939
2,Parser,0.766277,0.693353,0.727994,0.997921


In [9]:
rows = []
for c in test_collections_sc:
    d = compute_metrics_for_relations(c)
    rows.append(d)
pd.DataFrame(rows)

,algo,recall,precision,f1_score,accuracy
0,RNN,0.797853,0.785903,0.791833,0.995436
1,Stacked_Model,0.719141,0.816244,0.764622,0.995183
2,Parser,0.772809,0.808232,0.790123,0.995533
